In [ ]:
# Install related tools
%pip install bertviz jupyterlab ipywidgets

In [ ]:
# Import related packets and def function
from transformers import BertModel
from bertviz import head_view, model_view
from bertviz.neuron_view import show
import tensorflow as tf
import sys
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
from transformers import BertTokenizer, BertModel

def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))
clear_output()

def pred_sentence(model, sentence, tokenizer):
  inputs = tokenizer.encode_plus(sentence, return_tensors='pt', add_special_tokens=True)
  model = model.to(device)
  inputs = inputs.to(device)
  with torch.no_grad():
      outputs = model(**inputs)
  logits = outputs.logits
  probabilities = softmax(logits, dim=1).squeeze().tolist()

  print("Predicted Probabilities:")
  for class_idx, prob in enumerate(probabilities):
      print(f"Class {class_idx}: {prob:.4f}")

def get_viz(model, sentence, viz_tokenizer):
  inputs = viz_tokenizer.encode_plus(sentence, return_tensors='pt', add_special_tokens=True)
  token_type_ids = inputs['token_type_ids']
  input_ids = inputs['input_ids']
  viz_model  = BertModel.from_pretrained(PRETRAINED_MODEL_NAME, output_attentions=True, state_dict=model.state_dict())
  attention = viz_model(input_ids, token_type_ids=token_type_ids)[-1]
  input_id_list = input_ids[0].tolist()
  tokens = viz_tokenizer.convert_ids_to_tokens(input_id_list)
  call_html()
  head_view(attention, tokens)

In [ ]:
# Load model and device setting
loaded_model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
num_classes = 2
loaded_model.config.num_labels = num_classes
loaded_model.classifier = torch.nn.Linear(loaded_model.config.hidden_size, num_classes)
checkpoint = torch.load('fine_tuned_bert_model.pth', map_location=device)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_tokenizer = checkpoint['tokenizer']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loaded_model = loaded_model.to(device)
loaded_model.eval()

In [ ]:
# For the input sentence, output it's predition and attention.
input_sentence = "你是邪惡和超級棒且善良的人"
pred_sentence(loaded_model, input_sentence, loaded_tokenizer)
get_viz(loaded_model, input_sentence, loaded_tokenizer)